In [ ]:
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt 
import numpy as np
import cv2
from config import *

CATEGORIES_REV = {
    0: 'alert', 1: 'button', 2: 'card', 3:'checkbox_checked',
    4: 'checkbox_unchecked', 5: 'chip', 6: 'data_table', 7:'dropdown_menu',
    8: 'floating_action_button', 9: 'grid_list', 10: 'image', 11: 'label', 12: 'menu',
    13: 'radio_button_checked', 14: 'radio_button_unchecked', 15: 'slider', 16: 'switch_disabled',
    17: 'switch_enabled', 18: 'text_area', 19: 'text_field', 20: 'tooltip'
}

In [ ]:
class InputDate:
    __slots__ = ['images', '__index', 'labels', '_size']
    shape = (224, 224, 1)
    
    def __init__(self, _size: int) -> None:
        self.images = np.empty((_size,) + self.shape, dtype='float16')
        self.labels = np.empty(_size, dtype='uint8')
        
        self.__index = 0
        self._size = _size
    
    def add_img(self, _path: str, _cat: int) -> None:
        p = f"data/train/{CATEGORIES_REV[_cat]}/{_path}"
        img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
        img = cv2.bitwise_not(img)
        
        self.images[self.__index] = \
            np.asarray(img).astype('float32').reshape(self.shape) / 255
        self.labels[self.__index] = _cat
        self.__index += 1
        
        if self.__index == self._size:
            self.__done()
            return
            
        print(f"[+] Progress: {self.__index}/{self._size}")
    
    def __done(self):
        self.labels = to_categorical(self.labels)
        print(f"[+] DONE {self._size}/{self._size}")
    
    def show(self, _i: int) -> None:
        plt.imshow(self.images[_i], cmap='gray')

In [ ]:
import pandas as pd

train_input_f = pd.read_csv('data/processed/train_set.csv')

td = InputDate(len(train_input_f))

for i, _path, _cat in train_input_f.values:
    td.add_img(_path, _cat)

In [ ]:
td.show(14628)

In [ ]:
test_input_f = pd.read_csv('data/processed/test_set.csv')

test_d = InputDate(len(test_input_f))

for i, _path, _cat in test_input_f.values:
    test_d.add_img(_path, _cat)

In [ ]:
test_d.show(345)

In [ ]:
from tensorflow.keras import layers, models

h_activ, f_activ = 'relu', 'softmax'
kernel_s, pool_s = (3, 3), (2, 2)

model = models.Sequential()
model.add(layers.Conv2D(32, kernel_s, activation=h_activ, input_shape=td.shape))
model.add(layers.MaxPooling2D(pool_size=pool_s))
model.add(layers.Conv2D(64, kernel_s, activation=h_activ))
model.add(layers.MaxPooling2D(pool_size=pool_s))
model.add(layers.Conv2D(64, kernel_s, activation=h_activ))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation=h_activ))
model.add(layers.Dense(COUNT, activation=f_activ))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(td.images, td.labels, epochs=COUNT, batch_size=64, validation_split=0.2)
model.save('model_v0.1.h5')

In [ ]:
# from tensorflow.keras.models import load_model
# 
# model_test = load_model('model_v0.1.h5')
# 
# test_loss, test_acc = model_test.evaluate(test_d.images, test_d.labels)
# print(f'Test accuracy: {test_acc}')